# Test Run 12 Mini Dataset Exploration w/ Word-level Segments
In this notebook, we implement our decision of switching to word as the segmentation unit. See weekly progress report of 18Fall, week 6. 



In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))
debug_mode=1 # if not in debug mode, please set debug_mode=0, find more info about debug mode in test run 9 maybe?
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

In [4]:
import os
os.environ['SNORKELDB'] = "postgresql:///abstractnetword"
print(os.environ['SNORKELDB'])

from snorkel import SnorkelSession
from snorkel.parser import TSVDocPreprocessor
session = SnorkelSession()

# # Here, we just set how many documents we'll process for automatic testing- you can safely ignore this!
n_docs = 1000 # this is the upper limit of number of docs
doc_preprocessor = TSVDocPreprocessor('data/slim-s2-corpus-00-1000', encoding="utf-8",max_docs=n_docs)

from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

# corpus_parser = CorpusParser(parser=Spacy())
# %time corpus_parser.apply(doc_preprocessor, count=n_docs)# ,parallelism=5)

from snorkel.models import Document, Sentence  # defined in context.py file

print("Documents:", session.query(Document).count())

postgresql:///abstractnetword
Clearing existing...
snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
clear finished!
Running UDF...
snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
[========================================] 100%

CPU times: user 19.7 s, sys: 465 ms, total: 20.1 s
Wall time: 27.9 s
Documents: 1000


In [5]:
from snorkel import SnorkelSession
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence

session = SnorkelSession()
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

docs = session.query(Document).all()
sents = session.query(Sentence).all()  # get all sentences from snorkel.db
n_max_corpus=0
for sent in sents:
    n_max_corpus=max(n_max_corpus,len(sent.words))
print("The longest sentence has "+str(n_max_corpus)+" tokens.")

train_sents = sents

Documents: 1000
Sentences: 6843
The longest sentence has 180 tokens.


In [6]:
from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import *
import datetime

from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
    
### NOTE: this extract_and_display is different from earlier test run notebook's version, whrere selected_split=0 will only extract train_segments. 
def extract_and_display(matcher,candidate_class,candidate_class_name,train_breakdown_map=None,dev_doc_breakdown_map=None,selected_split=0,is_print=True):  # split over train/dev/test but returns only train set
    for (i, sents) in ([(0,train_sents)] if selected_split==0 else ([(2,test_sents)] if selected_split==2 else [(1,dev_sents)])):
        %time matcher.apply(sents, split=i)
        printmd("**Split "+str(i)+" - number of candidates extracted: "+str(session.query(candidate_class).filter(candidate_class.split == i).count())+"**\n\n")
    
    train_cands = session.query(candidate_class).filter(candidate_class.split == selected_split).all()
    dev_cands = session.query(candidate_class).filter(candidate_class.split == 1).all()
    test_cands=session.query(candidate_class).filter(candidate_class.split==2).all()
    
    return train_cands,dev_cands,test_cands

Unigram = candidate_subclass('Unigram', ['unigram_cue'],values=['PP','MN','NULL'])
ngrams = Ngrams(n_max=1) 
ngram_matcher=NgramMatcher()
unigram_segment_extractor=CandidateExtractor(Unigram,[ngrams],[ngram_matcher])

%time unigram_segment_extractor.apply(train_sents, split=0)
unigram_segments=session.query(Unigram).filter(Unigram.split == 0).all()
print("len(train_segments)",len(unigram_segments))

Clearing existing...
snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
Running UDF...
snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
[========================================] 100%

CPU times: user 4min 33s, sys: 18.8 s, total: 4min 51s
Wall time: 9min 46s
len(train_segments) 176727


In [7]:
# print(Candidate.__dict__)
# cands=session.query(Candidate).filter(Candidate.id%100<5).all()
# session.query(Candidate).filter(Candidate.split==1).delete()
# print(len(cands))
# print(cands[0].__dict__)

In [35]:
from snorkel.lf_helpers import *
from util import get_candidate_text



segmentid=1234

segment=unigram_segments[segmentid]
# print((segment.get_contexts()[0].__dict__))
print(segment.get_parent().text[segment.unigram_cue.char_start:segment.unigram_cue.char_end+1])

unigram_psn=segment.unigram_cue.get_word_start()
print("unigram position?? ",unigram_psn)#.__dict__['words'])

its_pos_tag=segment.get_parent().__dict__['pos_tags'][unigram_psn]
print("its_pos_tag",its_pos_tag,"\n")

# its_surrounding_pos_tag=get_sourrounding_pos_tag(segment,3)
print("unigram pos tag",get_sourrounding_pos_tag(segment,1))
print("trigram pos tag",get_sourrounding_pos_tag(segment,3))
print("5gram pos tag",get_sourrounding_pos_tag(segment,5))



# print("unigram\t",segment,"\n")  # Unigram(Span("b'time'", sentence=1006, chars=[33,36], words=[6,6])) 
# print("sent id\t",segment.get_parent().id,"\n")
# print("sentence all info\t",segment.get_parent(),"\n")
# print("sentence_dict\t",segment.get_parent().__dict__,"\n")

# print("[TODO]this unigram pos tag in sent ",segment.__dict__,"\n")
# print("[TODO]this unigram position in sent ",get_pos_tag_string(segment),"\n")

# print("all sentences in the doc\t",segment.get_parent().get_parent().sentences,"\n")
# print("this sentence pos in doc\t",segment.get_parent().get_parent().sentences.index(train_segments[docid].get_parent()),"\n")
# print("doc name\t", segment.get_parent().get_parent().name)



is
unigram position??  19
its_pos_tag VBZ 

unigram pos tag VBZ
trigram pos tag NNP VBZ JJR
5gram pos tag JJ NNP VBZ JJR IN


In [47]:
from LF.util_common_default_categorical import *


# TODO get_surrounding_words maybe group segments back to doc level from  Sentence has words in its __dict__ 

def get_surrounding_words(segment,window_size=3):
    pos_tag_list=segment.get_parent().__dict__['pos_tags']
    unigram_psn=segment.unigram_cue.get_word_start()
    half_window_size=int((window_size-1)/2)
    return " ".join(pos_tag_list[max(0,unigram_psn-half_window_size):min(len(pos_tag_list),unigram_psn+half_window_size+1)])

def create_POS_LFs_by_window(pair,lf_prefix):
    
    # pair = (pattern, window_size, label)
    pattern, window_size, label = pair
    
    def func(c):
        window_text=get_surrounding_words(c,window_size)
        return label if pattern in window_text else 0

    func.__name__=create_function_name(lf_prefix,pattern+str(window_size))
    return func

    
purpose_pos_tag_list=[("NN IN DT",PURPOSE_LABEL)] # cue: task for the
# purpose_pos_tag_list+=[("NN TO DT",PURPOSE_LABEL)] # cue: solution to the
purpose_pos_tag_list+=[("NNS IN DT",PURPOSE_LABEL)] # cue: tasks for the 
# purpose_pos_tag_list+=[("NNS TO DT",PURPOSE_LABEL)] # cue: solutions for the 
purpose_pos_tag_list+=[("TO VB",PURPOSE_LABEL)] # cue: to solve
purpose_pos_tag_list+=[("DT NN IN",PURPOSE_LABEL)] # cue: the problem of 
purpose_pos_tag_list+=[("IN VBG",PURPOSE_LABEL)] # cue: for improving
purpose_pos_tag_list+=[("DT VBZ",PURPOSE_LABEL)] # cue: This enables

purpose_LFs=[create_POS_LFs_by_window((pair[0],window_size,pair[1]),"purpose") for pair in purpose_pos_tag_list for window_size in [3,5,7,9]]

mechanism_pos_tag_list=[("PRP VBP",MECHANISM_LABEL)] # cue: we propose/present/...
mechanism_pos_tag_list+=[("PRP RB VBP",MECHANISM_LABEL)] # cue: we further/also propose/present/...
mechanism_pos_tag_list+=[("DT VBN NN",MECHANISM_LABEL)] # cue: the proposed method/methods
mechanism_pos_tag_list+=[("VBZ VBN",MECHANISM_LABEL)] # cue: is proposed
mechanism_pos_tag_list+=[("VBD VBN",MECHANISM_LABEL)] # cue: was proposed
mechanism_pos_tag_list+=[("VBP VBN",MECHANISM_LABEL)] # cue: are proposed
mechanism_pos_tag_list+=[("DT NN VBZ",MECHANISM_LABEL)] # cue: this/the paper proposes/presents/...
mechanism_pos_tag_list+=[("DT NN VBN",MECHANISM_LABEL)] # cue: the/this paper proposed/presented/...

mechanism_LFs=[create_POS_LFs_by_window((pair[0],window_size,pair[1]),"mechanism") for pair in mechanism_pos_tag_list for window_size in [3,5,7,9]]

### Section 2: Create generative labels for Purpose on train set

Any LFs for other aspects will be grouped into `neg_for_purpose_LFs`

In [37]:
from snorkel.lf_helpers import *
from snorkel.annotations import LabelAnnotator

print([lf.__name__ for lf in mechanism_LFs])
# from LF.util_purpose_default_categorical import common_LFs,purpose_LFs   # neg_LFs are reverse LFs for Background, Mechanism, Method, and Finding
# from LF.util_mechanism_default_categorical import mechanism_LFs
# from LF.util_background_default_categorical import background_LFs
# from LF.util_method_default_categorical import method_LFs
# from LF.util_finding_default_categorical import finding_LFs

# print("common_LFs",len(common_LFs))
# print("purpose_LFs",len(purpose_LFs))
# print("mechanism_LFs",len(mechanism_LFs))
# # print([lf.__name__ for lf in mechanism_LFs])
# print("method_LFs",len(method_LFs))
# print("background_LFs",len(background_LFs))
# print("finding_LFs",len(finding_LFs))


['LF_mechanism_NN_IN_DT3', 'LF_mechanism_NN_IN_DT5', 'LF_mechanism_NN_IN_DT7', 'LF_mechanism_NN_IN_DT9', 'LF_mechanism_NNS_IN_DT3', 'LF_mechanism_NNS_IN_DT5', 'LF_mechanism_NNS_IN_DT7', 'LF_mechanism_NNS_IN_DT9', 'LF_mechanism_TO_VB3', 'LF_mechanism_TO_VB5', 'LF_mechanism_TO_VB7', 'LF_mechanism_TO_VB9', 'LF_mechanism_DT_NN_IN3', 'LF_mechanism_DT_NN_IN5', 'LF_mechanism_DT_NN_IN7', 'LF_mechanism_DT_NN_IN9', 'LF_mechanism_IN_VBG3', 'LF_mechanism_IN_VBG5', 'LF_mechanism_IN_VBG7', 'LF_mechanism_IN_VBG9', 'LF_mechanism_DT_VBZ3', 'LF_mechanism_DT_VBZ5', 'LF_mechanism_DT_VBZ7', 'LF_mechanism_DT_VBZ9', 'LF_mechanism_DT_NN_IN3', 'LF_mechanism_DT_NN_IN5', 'LF_mechanism_DT_NN_IN7', 'LF_mechanism_DT_NN_IN9']


### Section 3: Repeat Section 2 for all other 4 aspects! 

In [55]:
from snorkel.learning import GenerativeModel
from util import get_candidate_text
import matplotlib.pyplot as plt
import datetime

# maybe write a cids_query session.query(Candidate.id).filter(Candidate.split == split) from def apply(self, split=0, key_group=0, replace_key_set=True, cids_query=None,
#         **kwargs):
def get_L_train(LFs,parallelism=1,split=0):
    L_train=None
    labeler=None
    np.random.seed(1701)
    labeler = LabelAnnotator(lfs=LFs)
    print(datetime.datetime.now())
    L_train = labeler.apply(split=split,parallelism=parallelism)# ,cids_query=session.query(Candidate.id).filter(Candidate.get_parent().id %10==1))
    print(datetime.datetime.now())
    print(type(L_train))
    print(L_train.shape)
    printmd("**Total non_overlapping_coverage on L_train (percentage of labelled over all)**  "+str(L_train.non_overlapping_coverage()))
    return L_train

def get_train_marginals(L_train,step_size_nominator=0.1,epochs=100,threads=1,class_prior=False,decay=1.0):
    try:
        gen_model=None
        print(datetime.datetime.now())
        gen_model = GenerativeModel(lf_propensity=True,class_prior=class_prior)
#         gen_model.train(L_train, cardinality=3, epochs=100, decay=0.95, step_size=step_size_nominator/L_train.shape[0], reg_param=0.0,verbose=False,threads=threads) # try step_size = 1/ next time? 
        gen_model.train(L_train, cardinality=3, epochs=epochs, decay=decay,step_size=step_size_nominator/1e2,reg_param=1e-6,verbose=True,threads=threads)
        print("training finished ")
        print(datetime.datetime.now())
        print("Finished training generative model, now checking performance against development set labels...")
        print("Calculating marginals!")
        train_marginals = gen_model.marginals(L_train)  # the marginal probability of each candidate being True
#         assert np.all(train_marginals.sum(axis=1) - np.ones(train_marginals.shape[0]) < 1e-10)
        print("no exception")
        return train_marginals
    except:
        print("exception! ")
        return train_marginals


In [56]:
L_train=get_L_train(purpose_LFs+mechanism_LFs)

snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
2018-10-04 12:59:03.371626
snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
cids_count 176727
key_group 0
Clearing existing...
snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
Running UDF...
snorkel_conn_string postgresql:///abstractnetword
if snorkel_postgres? True
[========================================] 100%

2018-10-04 13:20:30.199058
<class 'snorkel.annotations.csr_LabelMatrix'>
(176727, 56)


**Total non_overlapping_coverage on L_train (percentage of labelled over all)**  0.4717106044916736

In [90]:
print(len(set(purpose_LFs+mechanism_LFs)))
print(L_train.shape)

60
(176727, 56)


In [58]:
import pandas as pd 
pd.set_option('display.max_rows', 500)

L_train.lf_stats(session)

,j,Coverage,Overlaps,Conflicts
LF_purpose_NN_IN_DT3,0,0.015883,0.015883,0.002405
LF_purpose_NN_IN_DT5,1,0.047650,0.047650,0.007475
LF_purpose_NN_IN_DT7,2,0.077492,0.077492,0.012052
LF_purpose_NN_IN_DT9,3,0.105292,0.090360,0.017156
LF_purpose_NNS_IN_DT3,4,0.006818,0.006818,0.000809
LF_purpose_NNS_IN_DT5,5,0.020455,0.020455,0.002394
LF_purpose_NNS_IN_DT7,6,0.033843,0.033843,0.003921
LF_purpose_NNS_IN_DT9,7,0.046801,0.038823,0.005777
LF_purpose_TO_VB3,8,0.023799,0.023799,0.005223
LF_purpose_TO_VB5,9,0.046863,0.046863,0.010134


In [64]:
from snorkel.utils import sparse_nonzero
import scipy.sparse as sparse

L_nonzero=sparse_nonzero(L_train)
not_all_zero_row_idx=sorted(set(sparse.find(L_nonzero)[0]))

print(type(L_train[not_all_zero_row_idx]))
print("not_all_zero_row_idx has ",len(not_all_zero_row_idx))

L_train_trim=L_train[not_all_zero_row_idx]
print(L_train_trim.shape)

L_train_segments=[unigram_segments[i] for i in not_all_zero_row_idx]


<class 'snorkel.annotations.csr_AnnotationMatrix'>
not_all_zero_row_idx has  83364
(83364, 56)


In [65]:
dense=L_train_trim.todense()
print([(dense==i).sum() for i in range(4)])

[4396621, 183014, 88749, 0]


In [93]:
# L_train=get_L_train(common_LFs+purpose_LFs+mechanism_LFs+background_LFs+method_LFs+finding_LFs)
train_marginals=get_train_marginals(L_train,threads=18,epochs=100)

print(train_marginals)
print(train_marginals.sum(axis=1))
assert np.all(train_marginals.sum(axis=1) - np.ones(train_marginals.shape[0]) < 1e-10)

winHighlight=train_marginals.argmax(axis=1)
print("PP",(winHighlight==0).sum())
print("MN",(winHighlight==1).sum())
print("NULL",(winHighlight==2).sum())


2018-10-04 16:30:40.324706
Compiles variable matrix
LF label variables -- initial loop to set all variables over 0
LF label variables -- initial loop to set all variables over 10000
LF label variables -- initial loop to set all variables over 20000
LF label variables -- initial loop to set all variables over 30000
LF label variables -- initial loop to set all variables over 40000
LF label variables -- initial loop to set all variables over 50000
LF label variables -- initial loop to set all variables over 60000
LF label variables -- initial loop to set all variables over 70000
LF label variables -- initial loop to set all variables over 80000
LF label variables -- initial loop to set all variables over 90000
LF label variables -- initial loop to set all variables over 100000
LF label variables -- initial loop to set all variables over 110000
LF label variables -- initial loop to set all variables over 120000
LF label variables -- initial loop to set all variables over 130000
LF label v

FACTOR 0: EPOCH #1
Current stepsize = 0.001
Learning epoch took 20.489 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  0.8931519718364687

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  1.610017963992449

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  1.7179543430574484

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.6309239799166404

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  0.34220585955501204

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  1.0771388995496094

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fa

FACTOR 0: EPOCH #2
Current stepsize = 0.001
Learning epoch took 20.382 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  0.7469436136736035

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  1.4516686609094087

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  1.7335641487915747

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.6555575703155196

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  0.2501376491204516

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  0.8399194182345276

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fa

FACTOR 0: EPOCH #3
Current stepsize = 0.001
Learning epoch took 19.003 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  0.8976606519281785

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  1.4252124746921755

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  1.6470353762703478

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.441098961689382

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  0.528998022670744

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  0.9855950385300734

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fals

FACTOR 0: EPOCH #4
Current stepsize = 0.001
Learning epoch took 21.391 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  1.1243059173706904

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  1.562683092806899

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  1.7694369383236215

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.3936263498490356

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  0.5398113419895713

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  1.1482226421179214

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fal

...

FACTOR 0: EPOCH #96
Current stepsize = 0.001
Learning epoch took 20.095 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  2.5559887335354916

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  3.140044259872851

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  2.9891309451894696

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.594371736310531

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  2.1264364385162526

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  2.728861035320636

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fals

FACTOR 0: EPOCH #97
Current stepsize = 0.001
Learning epoch took 19.216 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  2.6450784473734035

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  3.1919309775082136

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  2.998058229482651

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.544821063040821

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  2.1316861560120133

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  2.727908725708044

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fals

FACTOR 0: EPOCH #98
Current stepsize = 0.001
Learning epoch took 18.564 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  2.5861649204520103

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  3.150825484761396

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  3.0139832284360337

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.5822669038844237

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  2.194919250085022

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  2.7429421965025136

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fal

FACTOR 0: EPOCH #99
Current stepsize = 0.001
Learning epoch took 18.396 sec.
Weights:
    weightId: 0
        isFixed: True
        weight:  1.0

    weightId: 1
        isFixed: False
        weight:  2.6522470156610476

    weightId: 2
        isFixed: True
        weight:  1.0

    weightId: 3
        isFixed: False
        weight:  3.2087029893091787

    weightId: 4
        isFixed: True
        weight:  1.0

    weightId: 5
        isFixed: False
        weight:  3.056916230324896

    weightId: 6
        isFixed: True
        weight:  1.0

    weightId: 7
        isFixed: False
        weight:  1.5207148133324178

    weightId: 8
        isFixed: True
        weight:  1.0

    weightId: 9
        isFixed: False
        weight:  2.1271582011515187

    weightId: 10
        isFixed: True
        weight:  1.0

    weightId: 11
        isFixed: False
        weight:  2.7669721710547193

    weightId: 12
        isFixed: True
        weight:  1.0

    weightId: 13
        isFixed: Fa

FACTOR 0: DONE WITH LEARNING
Compiles variable matrix
LF label variables -- initial loop to set all variables over 0
L_coo.nnz 0
Compiles factor and ftv matrices
Factors over labeling function outputs
nfactors_for_lf is None?  False
optional_names for _compile_output_factors
nfactors_for_lf is None?  True
_compile finished!
training finished 
2018-10-04 17:09:44.450966
Finished training generative model, now checking performance against development set labels...
Calculating marginals!
no exception
[[0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 ...
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]
 [0.33333333 0.33333333 0.33333333]]
[1. 1. 1. ... 1. 1. 1.]
PP 160259
MN 16468
NULL 0


Let's check if the generative labels makes any sense. -> Seems okay. 

In [94]:
pair_list=list(zip(unigram_segments,list(train_marginals[:,0].reshape(-1))))

pair_list.sort(key = lambda t: t[1],reverse=True)

def get_candidate_text(candidate):
    cue=candidate.get_contexts()[0]
    return cue.sentence.text[cue.char_start:cue.char_end+1]

top_cutoff=30
print("top purpose words \n\n")
print("\n\n".join([str(ind)+"\tWord\t"+get_candidate_text(pair[0])+"\n"+"Sentence\t"+str(pair[0].get_parent().text)+"\t"+str(pair[1]) for ind,pair in enumerate(pair_list[:top_cutoff])]))


pair_list=list(zip(unigram_segments,list(train_marginals[:,1].reshape(-1))))
pair_list.sort(key = lambda t: t[1],reverse=True)

print("\n\n\ntop mechanism words \n\n")
print("\n\n".join([str(ind)+"\tWord\t"+get_candidate_text(pair[0])+"\n"+"Sentence\t"+str(pair[0].get_parent().text)+"\t"+str(pair[1]) for ind,pair in enumerate(pair_list[:top_cutoff])]))

top purpose words 


0	Word	control
Sentence	The proposed strategy considers the translational dynamics of the vehicle to design a control of the thrust vector that ensures safe navigation in presence of encountered obstacles.	1.0

1	Word	of
Sentence	The proposed strategy considers the translational dynamics of the vehicle to design a control of the thrust vector that ensures safe navigation in presence of encountered obstacles.	1.0

2	Word	the
Sentence	The proposed strategy considers the translational dynamics of the vehicle to design a control of the thrust vector that ensures safe navigation in presence of encountered obstacles.	1.0

3	Word	design
Sentence	The proposed strategy considers the translational dynamics of the vehicle to design a control of the thrust vector that ensures safe navigation in presence of encountered obstacles.	1.0

4	Word	a
Sentence	The proposed strategy considers the translational dynamics of the vehicle to design a control of the thrust vector that ensures

### Section 3: writing soft generative noisy labels

In [31]:
from collections import defaultdict
from util import get_candidate_text
import pandas as pd

# initialize some variables
# df = pd.DataFrame([], columns=['winningHighlight', 'paperID', 'globalPsn','content'])    
aggregated_doc=defaultdict(lambda:[])
aggregated_sent=defaultdict(lambda:[])
aspects_list=["Purpose","Mechanism","Others"]

for idx,train_segment in enumerate(train_segments):
    
    sent_id=train_segment.get_parent().id    
    winningHighlight=list(train_marginals[idx])
    aggregated_sent[sent_id]+=[(train_segment.segment_cue.char_start,train_segment.segment_cue.char_end,get_candidate_text(train_segment),winningHighlight)]
    
    doc_id=train_segment.get_parent().get_parent().name
    if doc_id not in aggregated_doc:
        aggregated_doc[doc_id]=train_segment.get_parent().get_parent().sentences
        

df_list=[]      
for doc_id in aggregated_doc:
    globalPsn=1
    for sent in aggregated_doc[doc_id]:
        sent_id=sent.id
        if sent_id not in aggregated_sent:
            raise ValueError('we have no info about this sent'+str(sent))
        aggregated_sent[sent_id]=sorted(aggregated_sent[sent_id], key=lambda x: x[0])
        for idx,pair in enumerate(aggregated_sent[sent_id]):
            text=pair[2]
            for idx2,token in enumerate(text.split()):
                if len(token.strip())==0:
                    continue
                df_list+=[[pair[3],doc_id,globalPsn,token.strip(".")]]
                globalPsn+=1
            if idx!=len(aggregated_sent[sent_id])-1:
                df_list+=[[pair[3],doc_id,globalPsn,","]]
            else:
                df_list+=[[pair[3],doc_id,globalPsn,"."]]
            globalPsn+=1
                
                
df=pd.DataFrame(df_list,columns=['winningHighlight', 'paperID', 'globalPsn','content'])

df.to_csv(open("data/annotations_train_5560_word-level_categorical.csv","w"))

In [14]:
docid=1
print("clause\t",train_segments[docid],"\n")
print("sent id\t",train_segments[docid].get_parent().id,"\n")
print("sentence\t",train_segments[docid].get_parent(),"\n")
print("sentence_dict\t",train_segments[docid].get_parent().__dict__,"\n")
print("all sentences in the doc\t",train_segments[docid].get_parent().get_parent().sentences,"\n")
print("this sentence pos in doc\t",train_segments[docid].get_parent().get_parent().sentences.index(train_segments[docid].get_parent()),"\n")
# print(docs[1].sentences)
print("doc name\t", train_segments[docid].get_parent().get_parent().name)

clause	 Segment(Span("b'cello'", sentence=19479, chars=[96,100], words=[15,15])) 

sent id	 19479 

sentence	 Sentence(Document 58cca85cc50f90cdd388252e,0,b'Abstract.\xe2\x80\x9cThe Watercourse Way\xe2\x80\x9d is a mostly-music interactive multimedia performance for violin, cello, percussion, and dancer.') 

sentence_dict	 {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x1165ea048>, 'stable_id': '58cca85cc50f90cdd388252e::sentence:0:126', 'dep_parents': [3, 3, 5, 3, 0, 12, 9, 9, 12, 12, 12, 5, 12, 13, 14, 14, 16, 16, 18, 18, 18, 5], 'pos_tags': ['DT', 'NNP', 'NNP', '-RRB-', 'VBZ', 'DT', 'RB', 'HYPH', 'NN', 'JJ', 'NN', 'NN', 'IN', 'NN', ',', 'NN', ',', 'NN', ',', 'CC', 'NN', '.'], 'abs_char_offsets': [0, 14, 26, 29, 31, 34, 36, 42, 43, 49, 61, 72, 84, 88, 94, 96, 101, 103, 113, 115, 119, 125], 'entity_cids': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'char_offsets': [0, 14, 26, 29, 31, 34, 36, 42,